# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import NonSequenceTunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


# Define different types of Keras models

In [2]:
lambda_regularization = 0.20

def RNNModel(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    model.add(LSTM(input_shape=input_shape))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model
    

def RULmodel_SN_6(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

init
super init


# Create Tunable Model

In [4]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]


#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*window_stride
modelRULSN = RULmodel_SN_6(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

#min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
min_max_scaler = None

tModel = NonSequenceTunableModel('ModelRUL_SN_1', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps, 
                                 data_scaler=min_max_scaler)

# Load Data

In [8]:
tModel.load_data(verbose=1, cross_validation_ratio=0)
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0
Loading data from memmory
Printing shapes

Training data (X, y)
(17731, 420)
(17731, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[  641.82    1589.7     1400.6    ...,   390.        39.05      23.411 ]
 [  642.15    1591.82    1403.14   ...,   392.        38.94      23.3353]
 [  642.35    1587.99    1404.2    ...,   392.        39.02      23.4999]
 [  642.35    1582.79    1401.87   ...,   392.        38.83      23.3506]
 [  642.37    1582.85    1406.22   ...,   392.        38.81      23.3092]]
[[ 125.]
 [ 125.]
 [ 125.]
 [ 125.]
 [ 125.]]
Testing data (X, y)
[[  641.71    1588.45    1395.42   ...,   393.        38.81      23.3552]
 [  642.72    1589.15    1400.69   ...,   391.        38.81      23.2618]
 [  642.86    1588.9     1408.72   ...,   395.        38.93      23.274 ]
 [  642.45    1593.88    1407.84   ...,   395.        38.58